In [176]:
import pandas as pd
import re

## SQL Read

In [177]:
# Leer el archivo .sql
with open('Query_polizas_gs.sql', 'r', encoding='utf-8') as file:
    sql_query = file.read()

## SQL Transformation

In [ ]:
# Define una expresión regular para encontrar todo lo que está entre '--' y '---'
patron_1 = r'--Sumery:(.*?)--Step 1:'
patron_2 = r'(Step|LC)(.*?)--'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron_2, sql_query, re.DOTALL)

# Se guardar los comentarios en un df y en una lista. 
list_documentation = []
df_documentation = pd.DataFrame ()
df_documentation_html = pd.DataFrame ()

for coincidencia in coincidencias:
    list_documentation.append(str(coincidencia))


# Eliminar caracteres especiales de cada elemento de la lista
list_documentation = [re.sub(r'(([(]))', '', comentario) for comentario in list_documentation]
list_documentation = [re.sub(r'(([)]))', '', comentario) for comentario in list_documentation]
list_documentation = [re.sub(r'(([,]))', '', comentario) for comentario in list_documentation]
list_documentation = [re.sub(r"'", '', comentario) for comentario in list_documentation]

# Se guardan los comentarios en un DataFrame
df_documentation['comentarios'] = list_documentation
df_documentation_html['Comment'] = df_documentation['comentarios'].str.split(':').str[1]

# Se crea la campo Order que contiene el orden secuencial de los comentarios
df_documentation_html['Order'] = range(1, len(df_documentation) + 1)

# Se crea el campo Clase que identifica la clase a la que pertenece cada comentario
df_documentation_html['Clase'] = df_documentation['comentarios'].str.split(':').str[0]
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('_', '.')
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('  ', ' ')


In [217]:
# Funcion para identificar a que linea pertenece cada comentario
def line_count(cadena, sql):
    line_tmp = []
    line = [i + 1 for i, linea in enumerate(sql.splitlines()) if cadena in linea]
    line_tmp.append(line)
    return line_tmp

# Se crea el campo line_number ejecuntando la función line_count
line_number = []

for elemento in df_documentation_html['Comment']:
    line_number.append(line_count(elemento, sql_query))

line_number = [re.sub(r'(([1]))', '', comentario) for comentario in line_number]    
df_documentation_html['line_number'] = line_number  




TypeError: expected string or bytes-like object, got 'list'

## HTML Creation

In [180]:
import pandas as pd

# Convertir el DataFrame a HTML sin índice
table_html = df_documentation_html.to_html(index=False, escape=False)

# Agregar clases específicas a las columnas
table_html = table_html.replace("<th>Clase</th>", '<th class="Clase">Clase</th>')
table_html = table_html.replace("<th>Comment</th>", '<th class="Comment">Comment</th>')

# Agregar clases a las celdas de la columna "Coment"
for i in range(len(df_documentation_html['Comment'])):
    table_html = table_html.replace('<td>', '<td class="Comment">', 1)

# Definir el HTML completo con estilos CSS
html_string = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Process Tabla de Datos</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #2c2c2c;
            color: #d3d3d3;
            text-align: left;
            padding: 20px;
        }}
        h2 {{
            color: #ffffff;
        }}
        table {{
            margin: auto;
            border-collapse: collapse;
            background-color: #333;
            color: #d3d3d3;
            text-align: left;
            table-layout: auto;  /* Ajusta el ancho de las columnas al contenido */
        }}
        th, td {{
            border-left: none;
            border-right: none;
            border-top: 0.5px solid #555555;
            border-bottom: 0.5px solid #555555;
            padding: 10px;
            text-align: left;
            
        }}
        th {{
            background-color: #444;
        }}
        .Clase {{
            min-width: 100px; /* Ajusta el ancho de la columna Ciudad */
        }}

    </style>
</head>
<body>
    <h2>Process Comments</h2>
    {table_html}
</body>
</html>
"""

# Guardar en un archivo
with open("tabla.html", "w", encoding="utf-8") as f:
    f.write(html_string)

print("Archivo HTML con estilo generado con éxito.")


Archivo HTML con estilo generado con éxito.
